<a href="https://colab.research.google.com/github/karimul/Riset-EBM/blob/main/Deep_Energy_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial 8: Deep Energy-Based Generative Models

![Status](https://img.shields.io/static/v1.svg?label=Status&message=Finished&color=green)

**Filled notebook:** 
[![View on Github](https://img.shields.io/static/v1.svg?logo=github&label=Repo&message=View%20On%20Github&color=lightgrey)](https://github.com/phlippe/uvadlc_notebooks/blob/master/docs/tutorial_notebooks/tutorial8/Deep_Energy_Models.ipynb)
[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/phlippe/uvadlc_notebooks/blob/master/docs/tutorial_notebooks/tutorial8/Deep_Energy_Models.ipynb)  
**Pre-trained models:** 
[![View files on Github](https://img.shields.io/static/v1.svg?logo=github&label=Repo&message=View%20On%20Github&color=lightgrey)](https://github.com/phlippe/saved_models/tree/main/tutorial8)
[![GoogleDrive](https://img.shields.io/static/v1.svg?logo=google-drive&logoColor=yellow&label=GDrive&message=Download&color=yellow)](https://drive.google.com/drive/folders/11ZI7x2sfCNtaZUNpe4v08YXWN870spXs?usp=sharing)  
**Recordings:** 
[![YouTube - Part 1](https://img.shields.io/static/v1.svg?logo=youtube&label=YouTube&message=Part%201&color=red)](https://youtu.be/E6PDwquBBQc)
[![YouTube - Part 2](https://img.shields.io/static/v1.svg?logo=youtube&label=YouTube&message=Part%202&color=red)](https://youtu.be/QJ94zuSQoP4)

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')

ROOT = "/content/drive/MyDrive/Colab Notebooks"
sample_dir = os.path.join(ROOT, 'deep-energy-models')
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)
os.chdir(sample_dir)

In [ ]:
!pip install torchmetrics[image]

In [ ]:
## Standard libraries
import os
import json
import math
import numpy as np 
import random

## Imports for plotting
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline 
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgb
import matplotlib
from mpl_toolkits.mplot3d.axes3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.reset_orig()

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
# Torchvision
import torchvision
from torchvision.datasets import MNIST, CIFAR10, CelebA
from torchvision import transforms
# PyTorch Lightning
try:
    import pytorch_lightning as pl
except ModuleNotFoundError: # Google Colab does not have PyTorch Lightning installed by default. Hence, we do it here if necessary
    !pip install pytorch-lightning==1.4.9
    import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

## Torchmetrics
from torchmetrics import IS, FID

# Path to the folder where the datasets are/should be downloaded (e.g. CIFAR10, MNIST)
DATASET_PATH = "/content/data"
DATASET = "CIFAR10"
BATCH_SIZE = 128

# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = "runs"
CHECKPOINT_FILE = "CELEBA.ckpt"

#Training hyperparameter
TRAINING_STEPS = 60
TRAINING_STEP_SIZE = 10
MAX_EPOCHS = 20
DIVERGENCE = 'contrastive_divergence'

# Setting the seed
pl.seed_everything(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

# Initialize IS and FID
inception_metrics = IS().to(device, non_blocking=True)
fid_metrics = FID(feature=2048).to(device, non_blocking=True)

In [ ]:
if DATASET == 'CELEBA':
    !mkdir -p /content/data
    %cp -av '/content/drive/MyDrive/Colab Notebooks/improved_contrastive_divergence.v6/data/celeba' /content/data

We also have pre-trained models that we download below.

In [ ]:
# import urllib.request
# from urllib.error import HTTPError
# # Github URL where saved models are stored for this tutorial
# base_url = "https://raw.githubusercontent.com/phlippe/saved_models/main/tutorial8/"
# # Files to download
# pretrained_files = ["MNIST.ckpt", "tensorboards/events.out.tfevents.MNIST"]

# # Create checkpoint path if it doesn't exist yet
# os.makedirs(CHECKPOINT_PATH, exist_ok=True)

# # For each file, check whether it already exists. If not, try downloading it.
# for file_name in pretrained_files:
#     file_path = os.path.join(CHECKPOINT_PATH, file_name)
#     if "/" in file_name:
#         os.makedirs(file_path.rsplit("/",1)[0], exist_ok=True)
#     if not os.path.isfile(file_path):
#         file_url = base_url + file_name
#         print(f"Downloading {file_url}...")
#         try:
#             urllib.request.urlretrieve(file_url, file_path)
#         except HTTPError as e:
#             print("Something went wrong. Please try to download the file from the GDrive folder, or contact the author with the full output including the following error:\n", e)

# Energy Models


## Dataset


In [ ]:
class MNISTDataModule(pl.LightningDataModule):
    def __init__(self, data_dir: str = DATASET_PATH):
        super().__init__()
        self.data_dir = data_dir
        # Transformations applied on each image => make them a tensor and normalize between -1 and 1
        self.transform = transforms.Compose(
              [
                  transforms.ToTensor(),
                  transforms.Normalize((0.5,), (0.5,)),
              ]
          )
        self.dims = (1, 28, 28)

    def prepare_data(self):
        # download
        MNIST(self.data_dir, train=True, download=True)
        MNIST(self.data_dir, train=False, download=True)

    def setup(self, stage=None):
        
        # Loading the training dataset. We need to split it into a training and validation part
        if stage == "fit" or stage is None:
          self.mnist_train = MNIST(self.data_dir, train=True, transform=self.transform)
        
        # Assign test dataset for use in dataloader(s)
        if stage == "test" or stage is None:
          self.mnist_test = MNIST(self.data_dir, train=False, transform=self.transform)

    def train_dataloader(self):
        return data.DataLoader(self.mnist_train, batch_size=BATCH_SIZE, shuffle=True,  drop_last=True)

    def val_dataloader(self):
        return data.DataLoader(self.mnist_test, batch_size=BATCH_SIZE)

In [ ]:
class CIFAR10DataModule(pl.LightningDataModule):
    def __init__(self, data_dir: str = DATASET_PATH):
        super().__init__()
        self.data_dir = data_dir
        # Transformations applied on each image => make them a tensor and normalize between -1 and 1
        self.transform = transforms.Compose(
              [
                  transforms.ToTensor(),
                  transforms.Normalize((0.5,0.5,0.5,), (0.5,0.5,0.5,)),
              ]
          )
        self.dims = (3, 32, 32)
    def prepare_data(self):
        # download
        CIFAR10(self.data_dir, train=True, download=True)
        CIFAR10(self.data_dir, train=False, download=True)

    def setup(self, stage = None):      
        # Loading the training dataset. We need to split it into a training and validation part
        # Assign train/val datasets for use in dataloaders
        if stage == "fit" or stage is None:
            self.cifar_train = CIFAR10(self.data_dir, train=True, transform=self.transform)
        
        if stage == "test" or stage is None:
            self.cifar_test = CIFAR10(self.data_dir, train=False, transform=self.transform)

    def train_dataloader(self):
        return data.DataLoader(self.cifar_train, batch_size=BATCH_SIZE, shuffle=True,  drop_last=True)

    def val_dataloader(self):
        return data.DataLoader(self.cifar_test, batch_size=BATCH_SIZE)

In [ ]:
class CelebADataModule(pl.LightningDataModule):
    def __init__(self, data_dir: str = DATASET_PATH):
        super().__init__()
        self.data_dir = data_dir
        # Transformations applied on each image => make them a tensor and normalize between -1 and 1
        self.transform = transforms.Compose(
              [
                  transforms.ToTensor(),
                  transforms.Normalize((0.5,0.5,0.5,), (0.5,0.5,0.5,)),
              ]
          )
        self.dims = (3, 32, 32)
    def prepare_data(self):
        # download
        CelebA(self.data_dir, train=True, download=True)
        CelebA(self.data_dir, train=False, download=True)

    def setup(self, stage = None):      
        # Loading the training dataset. We need to split it into a training and validation part
        # Assign train/val datasets for use in dataloaders
        if stage == "fit" or stage is None:
            self.celeba_train = CelebA(self.data_dir, train=True, transform=self.transform)
        
        if stage == "test" or stage is None:
            self.celeba_test = CelebA(self.data_dir, train=False, transform=self.transform)

    def train_dataloader(self):
        return data.DataLoader(self.celeba_train, batch_size=BATCH_SIZE, shuffle=True,  drop_last=True)

    def val_dataloader(self):
        return data.DataLoader(self.celeba_test, batch_size=BATCH_SIZE)

## CNN Model


In [ ]:
class Swish(nn.Module):    
    def forward(self, x):
        return x * torch.sigmoid(x)


class CNNModel(nn.Module):
    def __init__(self, hidden_features=32, out_dim=1, **kwargs):
        super().__init__()
        # We increase the hidden dimension over layers. Here pre-calculated for simplicity.
        c_hid1 = hidden_features//2
        c_hid2 = hidden_features
        c_hid3 = hidden_features*2
        
        # Series of convolutions and Swish activation functions
        self.cnn_layers = nn.Sequential(
                nn.Conv2d(1, c_hid1, kernel_size=5, stride=2, padding=4), # [16x16] - Larger padding to get 32x32 image
                Swish(),
                nn.Conv2d(c_hid1, c_hid2, kernel_size=3, stride=2, padding=1), #  [8x8]
                Swish(),
                nn.Conv2d(c_hid2, c_hid3, kernel_size=3, stride=2, padding=1), # [4x4]
                Swish(),
                nn.Conv2d(c_hid3, c_hid3, kernel_size=3, stride=2, padding=1), # [2x2]
                Swish(),
                nn.Flatten(),
                nn.Linear(c_hid3*4, c_hid3),
                Swish(),
                nn.Linear(c_hid3, out_dim)
        )

    def forward(self, x):
        x = self.cnn_layers(x).squeeze(dim=-1)
        return x

In [ ]:
class StandardCNN(nn.Module):
    def __init__(self, hidden_features=32, out_dim=1, **kwargs):
        super().__init__()
        # We increase the hidden dimension over layers. Here pre-calculated for simplicity.
        c_hid1 = hidden_features*2 #64
        c_hid2 = hidden_features*4 #128
        c_hid3 = hidden_features*8 #256
        c_hid4 = hidden_features*16 #512

        self.conv1 = nn.utils.spectral_norm(nn.Conv2d(3, c_hid1, 3, 1, 1))
        self.conv2 = nn.utils.spectral_norm(nn.Conv2d(c_hid1, c_hid1, 4, 2, 1))

        self.conv3 = nn.utils.spectral_norm(nn.Conv2d(c_hid1, c_hid2, 3, 1, 1))
        self.conv4 = nn.utils.spectral_norm(nn.Conv2d(c_hid2, c_hid2, 4, 2, 1))

        self.conv5 = nn.utils.spectral_norm(nn.Conv2d(c_hid2, c_hid3, 3, 1, 1))
        self.conv6 = nn.utils.spectral_norm(nn.Conv2d(c_hid3, c_hid3, 4, 2, 1))

        self.conv7 = nn.utils.spectral_norm(nn.Conv2d(c_hid3, c_hid4, 3, 1, 1))

        self.pool = nn.MaxPool2d(2, 2)
        # self.act = nn.LeakyReLU(negative_slope=0.1, inplace=True)
        self.act = Swish()
        self.dense = nn.utils.spectral_norm(nn.Linear(c_hid4 * 4 * 4, out_dim))

    def forward(self, x):

        x = self.act(self.conv1(x))
        x = self.act(self.conv2(x))
        x = self.act(self.conv3(x))
        x = self.act(self.conv4(x))
        x = self.act(self.conv5(x))
        x = self.act(self.conv6(x))
        x = self.act(self.conv7(x))

        x = self.dense(x.view(x.shape[0], -1))

        return x

In [ ]:
class ConvBNReLU(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size=3, stride=1, groups=1):
        if isinstance(kernel_size, tuple):
            padding = (max(kernel_size) - 1) // 2
        else:
            padding = (kernel_size - 1) // 2

        super(ConvBNReLU, self).__init__()
        self.conv = nn.utils.spectral_norm(nn.Conv2d(
            in_planes,
            out_planes,
            kernel_size,
            stride,
            padding,
            groups=groups,
            bias=False,
        ))
        self.bn_normal = nn.BatchNorm2d(out_planes)
        self.bn_adversial = nn.BatchNorm2d(out_planes)
        self.act = nn.ReLU(inplace=True)

    def forward(self, x, adversial=False):
        x = self.conv(x)
        if adversial:
            x = self.bn_adversial(x)
        else:
            x = self.bn_normal(x)
        x = self.act(x)

        return x

class StandardBNCNN(nn.Module):
    def __init__(self, hidden_features=32, out_dim=1, **kwargs):
        super().__init__()
        c_hid1 = hidden_features*2 #64
        c_hid2 = hidden_features*4 #128
        c_hid3 = hidden_features*8 #256
        c_hid4 = hidden_features*16 #512

        self.conv1 = ConvBNReLU(3, c_hid1)
        self.conv2 = ConvBNReLU(c_hid1, c_hid1)
        self.conv3 = ConvBNReLU(c_hid1, c_hid2)
        self.conv4 = ConvBNReLU(c_hid2, c_hid2)
        self.conv5 = ConvBNReLU(c_hid2, c_hid3)
        self.conv6 = ConvBNReLU(c_hid3, c_hid3)
        self.conv7 = ConvBNReLU(c_hid3, c_hid4)
        self.conv8 = ConvBNReLU(c_hid4, c_hid4)
        self.max_pool = nn.MaxPool2d((2, 2))
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.utils.spectral_norm(nn.Linear(c_hid4, out_dim))

    def forward(self, x, adversial=False):
        x = self.conv1(x, adversial)
        x = self.conv2(x, adversial)
        x = self.max_pool(x)
        x = self.conv3(x, adversial)
        x = self.conv4(x, adversial)
        x = self.max_pool(x)
        x = self.conv5(x, adversial)
        x = self.conv6(x, adversial)
        x = self.max_pool(x)
        x = self.conv7(x, adversial)
        x = self.conv8(x, adversial)
        # (B, 512, 4, 4)

        x = self.avg_pool(x)
        x = torch.flatten(x, 1)

        x = self.fc(x)

        return x

In [ ]:
class ResBlock(nn.Module):
    def __init__(self, in_channel, out_channel, n_class=None, downsample=False):
        super().__init__()
        #add spectral normalization to all layers of the model
        self.conv1 = nn.utils.spectral_norm(
            nn.Conv2d(
                in_channel,
                out_channel,
                3,
                padding=1,
                bias=False if n_class is not None else True,
            )
        )

        self.conv2 = nn.utils.spectral_norm(
            nn.Conv2d(
                out_channel,
                out_channel,
                3,
                padding=1,
                bias=False if n_class is not None else True,
            ), std=1e-10, bound=True
        )

        self.class_embed = None

        if n_class is not None:
            class_embed = nn.Embedding(n_class, out_channel * 2 * 2)
            class_embed.weight.data[:, : out_channel * 2] = 1
            class_embed.weight.data[:, out_channel * 2 :] = 0

            self.class_embed = class_embed

        self.skip = None

        if in_channel != out_channel or downsample:
            self.skip = nn.Sequential(
                spectral_norm(nn.Conv2d(in_channel, out_channel, 1, bias=False))
            )

        self.downsample = downsample

    def forward(self, input, class_id=None):
        out = input

        out = self.conv1(out)

        if self.class_embed is not None:
            embed = self.class_embed(class_id).view(input.shape[0], -1, 1, 1)
            weight1, weight2, bias1, bias2 = embed.chunk(4, 1)
            out = weight1 * out + bias1

        out = F.leaky_relu(out, negative_slope=0.2)

        out = self.conv2(out)

        if self.class_embed is not None:
            out = weight2 * out + bias2

        if self.skip is not None:
            skip = self.skip(input)

        else:
            skip = input

        out = out + skip

        if self.downsample:
            out = F.avg_pool2d(out, 2)

        out = F.leaky_relu(out, negative_slope=0.2)

        return out

class ResNet(nn.Module):
    def __init__(self, n_class=None):
        super().__init__()
        #add spectral normalization to all layers of the model
        self.conv1 = nn.utils.spectral_norm(nn.Conv2d(3, 128, 3, padding=1), std=1)

        self.blocks = nn.ModuleList(
            [
                ResBlock(128, 128, n_class, downsample=True),
                ResBlock(128, 128, n_class),
                ResBlock(128, 256, n_class, downsample=True),
                ResBlock(256, 256, n_class),
                ResBlock(256, 256, n_class, downsample=True),
                ResBlock(256, 256, n_class),
            ]
        )

        self.linear = nn.Linear(256, 1)

    def forward(self, input, class_id=None):
        out = self.conv1(input)

        out = F.leaky_relu(out, negative_slope=0.2)

        for block in self.blocks:
            out = block(out, class_id)

        out = F.relu(out)
        out = out.view(out.shape[0], out.shape[1], -1).sum(2)
        out = self.linear(out)

        return out

## Sampling buffer


In [ ]:
class Sampler:

    def __init__(self, model, img_shape, sample_size, max_len=8192):
        """
        Inputs:
            model - Neural network to use for modeling E_theta
            img_shape - Shape of the images to model
            sample_size - Batch size of the samples
            max_len - Maximum number of data points to keep in the buffer
        """
        super().__init__()
        self.model = model
        self.img_shape = img_shape
        self.sample_size = sample_size
        self.max_len = max_len
        self.examples = [(torch.rand((1,)+img_shape)*2-1) for _ in range(self.sample_size)]

    def sample_new_exmps(self, steps=60, step_size=10):
        """
        Function for getting a new batch of "fake" images.
        Inputs:
            steps - Number of iterations in the MCMC algorithm
            step_size - Learning rate nu in the algorithm above
        """
        # Choose 95% of the batch from the buffer, 5% generate from scratch
        n_new = np.random.binomial(self.sample_size, 0.05)
        rand_imgs = torch.rand((n_new,) + self.img_shape) * 2 - 1
        old_imgs = torch.cat(random.choices(self.examples, k=self.sample_size-n_new), dim=0)
        inp_imgs = torch.cat([rand_imgs, old_imgs], dim=0).detach().to(device)

        # Perform MCMC sampling
        inp_imgs = Sampler.generate_contour_sgld_samples(self.model, inp_imgs, steps=steps, step_size=step_size)

        # Add new images to the buffer and remove old ones if needed
        self.examples = list(inp_imgs.to(torch.device("cpu")).chunk(self.sample_size, dim=0)) + self.examples
        self.examples = self.examples[:self.max_len]
        return inp_imgs

    @staticmethod
    def generate_sgld_samples(model, inp_imgs, steps=60, step_size=10, return_img_per_step=False):
        """
        Function for sampling images for a given model. 
        Inputs:
            model - Neural network to use for modeling E_theta
            inp_imgs - Images to start from for sampling. If you want to generate new images, enter noise between -1 and 1.
            steps - Number of iterations in the MCMC algorithm.
            step_size - Learning rate nu in the algorithm above
            return_img_per_step - If True, we return the sample at every iteration of the MCMC
        """
        # Before MCMC: set model parameters to "required_grad=False"
        # because we are only interested in the gradients of the input. 
        is_training = model.training
        model.eval()
        for p in model.parameters():
            p.requires_grad = False
        inp_imgs.requires_grad = True
        
        # Enable gradient calculation if not already the case
        had_gradients_enabled = torch.is_grad_enabled()
        torch.set_grad_enabled(True)
        
        # We use a buffer tensor in which we generate noise each loop iteration.
        # More efficient than creating a new tensor every iteration.
        noise = torch.randn(inp_imgs.shape, device=inp_imgs.device)
        
        # List for storing generations at each step (for later analysis)
        imgs_per_step = []
        
        # Loop over K (steps)
        for _ in range(steps):
            # Part 1: Add noise to the input.
            noise.normal_(0, 0.005)
            inp_imgs.data.add_(noise.data)
            inp_imgs.data.clamp_(min=-1.0, max=1.0)

            # Part 2: calculate gradients for the current input.
            out_imgs = -model(inp_imgs)
            out_imgs.sum().backward()
            inp_imgs.grad.data.clamp_(-0.03, 0.03) # For stabilizing and preventing too high gradients

            # Apply gradients to our current samples
            inp_imgs.data.add_(-step_size * inp_imgs.grad.data)
            inp_imgs.grad.detach_()
            inp_imgs.grad.zero_()
            inp_imgs.data.clamp_(min=-1.0, max=1.0)
            
            if return_img_per_step:
                imgs_per_step.append(inp_imgs.clone().detach())
        
        # Reactivate gradients for parameters for training
        for p in model.parameters():
            p.requires_grad = True
        model.train(is_training)
        
        # Reset gradient calculation to setting before this function
        torch.set_grad_enabled(had_gradients_enabled)

        if return_img_per_step:
            return torch.stack(imgs_per_step, dim=0)
        else:
            return inp_imgs
        
    @staticmethod
    def generate_cyclic_sgld_samples(model, inp_imgs, steps=60, step_size=10, return_img_per_step=False):
        """
        Function for sampling images for a given model. 
        Inputs:
            model - Neural network to use for modeling E_theta
            inp_imgs - Images to start from for sampling. If you want to generate new images, enter noise between -1 and 1.
            steps - Number of iterations in the MCMC algorithm.
            step_size - Learning rate nu in the algorithm above
            return_img_per_step - If True, we return the sample at every iteration of the MCMC
        """
        # Before MCMC: set model parameters to "required_grad=False"
        # because we are only interested in the gradients of the input. 
        is_training = model.training
        model.eval()
        for p in model.parameters():
            p.requires_grad = False
        inp_imgs.requires_grad = True
        
        # Enable gradient calculation if not already the case
        had_gradients_enabled = torch.is_grad_enabled()
        torch.set_grad_enabled(True)
        
        # We use a buffer tensor in which we generate noise each loop iteration.
        # More efficient than creating a new tensor every iteration.
        noise = torch.randn(inp_imgs.shape, device=inp_imgs.device)
        
        # List for storing generations at each step (for later analysis)
        imgs_per_step = []
        
        cycles=2
        # Loop over K (steps)
        for i in range(1, steps+1):            
            # Part 1: Add noise to the input.
            noise.normal_(0, 0.005)
            inp_imgs.data.add_(noise.data)
            inp_imgs.data.clamp_(min=-1.0, max=1.0)

            # Part 2: calculate gradients for the current input.
            out_imgs = -model(inp_imgs)
            out_imgs.sum().backward()
            inp_imgs.grad.data.clamp_(-0.03, 0.03) # For stabilizing and preventing too high gradients
            
            # Cyclical parameter
            sub_total = steps / cycles
            r_remainder = (i % sub_total) * 1.0 / sub_total
            cyc_lr = step_size * 5 / 2 * (math.cos(math.pi * r_remainder) + 1)
            
            # Apply gradients to our current samples
            inp_imgs.data.add_(-cyc_lr * inp_imgs.grad.data)
            inp_imgs.grad.detach_()
            inp_imgs.grad.zero_()
            inp_imgs.data.clamp_(min=-1.0, max=1.0)
            
            if return_img_per_step:
                imgs_per_step.append(inp_imgs.clone().detach())
        
        # Reactivate gradients for parameters for training
        for p in model.parameters():
            p.requires_grad = True
        model.train(is_training)
        
        # Reset gradient calculation to setting before this function
        torch.set_grad_enabled(had_gradients_enabled)

        if return_img_per_step:
            return torch.stack(imgs_per_step, dim=0)
        else:
            return inp_imgs
    
    @staticmethod
    def generate_replica_sgld_samples(model, inp_low_imgs, steps=60, step_size=10, return_img_per_step=False):
        """
        Function for sampling images for a given model. 
        Inputs:
            model - Neural network to use for modeling E_theta
            inp_imgs - Images to start from for sampling. If you want to generate new images, enter noise between -1 and 1.
            steps - Number of iterations in the MCMC algorithm.
            step_size - Learning rate nu in the algorithm above
            return_img_per_step - If True, we return the sample at every iteration of the MCMC
        """
        # Before MCMC: set model parameters to "required_grad=False"
        # because we are only interested in the gradients of the input. 
        is_training = model.training
        model.eval()
        for p in model.parameters():
            p.requires_grad = False
        
        inp_high_imgs = inp_low_imgs.clone()
        
        inp_low_imgs.requires_grad = True
        inp_high_imgs.requires_grad = True

        # Enable gradient calculation if not already the case
        had_gradients_enabled = torch.is_grad_enabled()
        torch.set_grad_enabled(True)
        
        # We use a buffer tensor in which we generate noise each loop iteration.
        # More efficient than creating a new tensor every iteration.
        noise_low = torch.randn(inp_low_imgs.shape, device=inp_low_imgs.device)
        noise_high = torch.randn(inp_low_imgs.shape, device=inp_low_imgs.device)
        
        # List for storing generations at each step (for later analysis)
        imgs_per_step = []
        
        #initialize resgld
        T_multiply = 3
        var = 0.1
        T = 1.0

        # Loop over K (steps)
        for _ in range(steps):
            # Part 1: Add noise to the low input.
            noise_low.normal_(0, 0.003)
            noise_high.normal_(0, 0.003 * T_multiply)
            inp_low_imgs.data.add_(noise_low.data)
            inp_high_imgs.data.add_(noise_high.data)
            inp_low_imgs.data.clamp_(min=-1.0, max=1.0)
            inp_high_imgs.data.clamp_(min=-1.0, max=1.0)

            # Part 2: calculate gradients for the current low input.
            out_low_imgs = -model(inp_low_imgs)
            out_high_imgs = -model(inp_high_imgs)
            out_low_imgs.sum().backward()
            out_high_imgs.sum().backward()
            inp_low_imgs.grad.data.clamp_(-0.03, 0.03) # For stabilizing and preventing too high gradients
            inp_high_imgs.grad.data.clamp_(-0.03, 0.03) # For stabilizing and preventing too high gradients

            # Apply gradients to our current samples
            inp_low_imgs.data.add_(-step_size * inp_low_imgs.grad.data)
            inp_high_imgs.data.add_(-step_size * inp_high_imgs.grad.data)
            inp_low_imgs.data.clamp_(min=-1.0, max=1.0)
            inp_high_imgs.data.clamp_(min=-1.0, max=1.0)

            dT = (1 / T) - (1 / (T * T_multiply))
            swap_rate = torch.exp(dT * (inp_low_imgs.grad.data - inp_high_imgs.grad.data - dT * var))
            intensity_r = 0.1

            inp_high_imgs.grad.detach_()
            inp_high_imgs.grad.zero_()
            inp_low_imgs.grad.detach_()
            inp_low_imgs.grad.zero_()

            if np.random.uniform(0, 1) < intensity_r * swap_rate.mean().item():
                inp_high_imgs, inp_low_imgs = inp_low_imgs, inp_high_imgs

            if return_img_per_step:
                imgs_per_step.append(inp_low_imgs.clone().detach())
            
        
        # Reactivate gradients for parameters for training
        for p in model.parameters():
            p.requires_grad = True
        model.train(is_training)
        
        # Reset gradient calculation to setting before this function
        torch.set_grad_enabled(had_gradients_enabled)

        if return_img_per_step:
            return torch.stack(imgs_per_step, dim=0)
        else:
            return inp_low_imgs

    @staticmethod
    def generate_contour_sgld_samples(model, inp_imgs, steps=60, step_size=10, return_img_per_step=False):
        """
        Function for sampling images for a given model. 
        Inputs:
            model - Neural network to use for modeling E_theta
            inp_imgs - Images to start from for sampling. If you want to generate new images, enter noise between -1 and 1.
            steps - Number of iterations in the MCMC algorithm.
            step_size - Learning rate nu in the algorithm above
            return_img_per_step - If True, we return the sample at every iteration of the MCMC
        """
        # Before MCMC: set model parameters to "required_grad=False"
        # because we are only interested in the gradients of the input. 
        is_training = model.training
        model.eval()
        for p in model.parameters():
            p.requires_grad = False
        inp_imgs.requires_grad = True
        
        # Enable gradient calculation if not already the case
        had_gradients_enabled = torch.is_grad_enabled()
        torch.set_grad_enabled(True)
        
        # We use a buffer tensor in which we generate noise each loop iteration.
        # More efficient than creating a new tensor every iteration.
        noise = torch.randn(inp_imgs.shape, device=inp_imgs.device)
        
        # List for storing generations at each step (for later analysis)
        imgs_per_step = []
        
        grad_mul = 1.0
        parts = 100
        J = parts - 1
        grad_mul = 1.
        Gcum = np.array(range(parts, 0, -1)) * 1.0 / sum(range(parts, 0, -1))
        decay_lr = 3e-1
        zeta = 0.75
        T = 1.0
        lower_bound, upper_bound = -7., 17.
        div_f = (upper_bound - lower_bound) / parts

        # Loop over K (steps)
        for i in range(steps):
            # Part 1: Add noise to the input.
            noise.normal_(0, 0.005)
            inp_imgs.data.add_(noise.data)
            inp_imgs.data.clamp_(min=-1.0, max=1.0)

            # Part 2: calculate gradients for the current input.
            out_imgs = -model(inp_imgs)
            out_imgs.sum().backward()
            inp_imgs.grad.data.clamp_(-0.03, 0.03) # For stabilizing and preventing too high gradients

            # lower_bound, upper_bound = torch.min(inp_imgs.grad.data).item() - 1, torch.max(inp_imgs.grad.data).item() + 1
            # div_f = (upper_bound - lower_bound) / parts
            grad_mul = 1 + zeta * T * (np.log(Gcum[J]) - np.log(Gcum[J-1])) / div_f
            J = min(max(int((inp_imgs.grad.data.mean() - lower_bound) / div_f + 1), 1), parts - 1)

            lr = min(decay_lr, 10./(i**0.8+100))

            Gcum[:J] = Gcum[:J] + lr * Gcum[J]**zeta * (-Gcum[:J])
            Gcum[J] = Gcum[J] + lr * Gcum[J]**zeta * (1 - Gcum[J])
            Gcum[(J+1):] = Gcum[(J+1):] + lr * Gcum[J]**zeta * (-Gcum[(J+1):])
            
            # Apply gradients to our current samples
            inp_imgs.data.add_(-step_size * grad_mul * inp_imgs.grad.data)
            inp_imgs.grad.detach_()
            inp_imgs.grad.zero_()
            inp_imgs.data.clamp_(min=-1.0, max=1.0)

            if return_img_per_step:
                imgs_per_step.append(inp_imgs.clone().detach())
        
        # Reactivate gradients for parameters for training
        for p in model.parameters():
            p.requires_grad = True
        model.train(is_training)
        
        # Reset gradient calculation to setting before this function
        torch.set_grad_enabled(had_gradients_enabled)

        if return_img_per_step:
            return torch.stack(imgs_per_step, dim=0)
        else:
            return inp_imgs

## Training algorithm


### Model

In [ ]:
class DeepEnergyModel(pl.LightningModule):
    
    def __init__(self, img_shape, batch_size, divergence='contrastive_divergence', alpha=0.1, lr=1e-4, beta1=0.0, **CNN_args):
        super().__init__()
        self.save_hyperparameters()
        
        self.cnn = CNNModel(**CNN_args) if img_shape[0] == 1 else StandardBNCNN(**CNN_args)
        
        self.sampler = Sampler(self.cnn, img_shape=img_shape, sample_size=batch_size)
        self.example_input_array = torch.zeros(1, *img_shape)
        self.last_real_imgs = torch.rand((batch_size,) + img_shape)
        self.last_fake_imgs = torch.rand((batch_size,) + img_shape)
        self.divergence = divergence
 
    def forward(self, x):
        z = self.cnn(x)
        return z

    def stable_exp(self, x):
        return torch.exp(torch.clip(x, min=-5., max=5.))

    def grad_exp(self, x):
        if self.divergence == 'reverse_kl':
            return - self.stable_exp(-x)
        elif self.divergence == 'kl':
            return 1 + x
        elif self.divergence == 'pearson_x2':
            return 2. * (self.stable_exp(x) - 1)
        elif self.divergence == 'jensen_shannon':
            return torch.log(torch.tensor(2.)) + x - torch.log(1 + self.stable_exp(x))
        elif self.divergence == 'squared_hellinger':
            return 1. - self.stable_exp(-0.5 * x)

    def conjugate_grad_exp(self, x):
        if self.divergence == 'reverse_kl':
            return -1. + x
        elif self.divergence == 'kl':
            return self.stable_exp(x)
        elif self.divergence == 'pearson_x2':
            return self.stable_exp(2. * x) - 1
        elif self.divergence == 'jensen_shannon':
            return -torch.log(torch.tensor(2.)) + torch.log(1 + self.stable_exp(x))
        elif self.divergence == 'squared_hellinger':
            return self.stable_exp(0.5 * x) - 1
        
    
    def configure_optimizers(self):
        # Energy models can have issues with momentum as the loss surfaces changes with its parameters. 
        # Hence, we set it to 0 by default. 
        optimizer = optim.Adam(self.parameters(), lr=self.hparams.lr, betas=(self.hparams.beta1, 0.999))
        scheduler = optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.97) # Exponential decay over epochs
        return [optimizer], [scheduler]

    def training_step(self, batch, batch_idx):
        # We add minimal noise to the original images to prevent the model from focusing on purely "clean" inputs
        real_imgs, _ = batch
        self.last_real_imgs = real_imgs.detach()
        small_noise = torch.randn_like(real_imgs) * 0.005
        real_imgs.add_(small_noise).clamp_(min=-1.0, max=1.0)
        
        # Obtain samples
        fake_imgs = self.sampler.sample_new_exmps(steps=TRAINING_STEPS, step_size=TRAINING_STEP_SIZE)
        self.last_fake_imgs = fake_imgs.detach()

        # Predict energy score for all images
        inp_imgs = torch.cat([real_imgs, fake_imgs], dim=0)
        real_out, fake_out = self.cnn(inp_imgs).chunk(2, dim=0)
        
        # Calculate losses
        reg_loss = 0.
        cdiv_loss = 0.
        f_div_loss = 0.
        if self.divergence == 'contrastive_divergence':
            reg_loss = self.hparams.alpha * (real_out ** 2 + fake_out ** 2).mean()
            cdiv_loss = fake_out.mean() - real_out.mean()
            loss = reg_loss + cdiv_loss
        else:
            f_div_loss = -(torch.mean(self.grad_exp(real_out)) + 
                                       torch.mean(self.conjugate_grad_exp(fake_out).detach() * fake_out) - 
                                       torch.mean(self.conjugate_grad_exp(fake_out)) - 
                                       torch.mean(fake_out) * torch.mean(self.conjugate_grad_exp(fake_out)).detach())
            loss = f_div_loss 
        
        # Logging
        self.log('loss', loss)
        self.log('loss_regularization', reg_loss)
        self.log('loss_contrastive_divergence', cdiv_loss)
        self.log('loss_f_divergence', f_div_loss)
        self.log('metrics_avg_real', real_out.mean())
        self.log('metrics_avg_fake', fake_out.mean())
        return loss

    def validation_step(self, batch, batch_idx):
        # For validating, we calculate the contrastive divergence between purely random images and unseen examples
        # Note that the validation/test step of energy-based models depends on what we are interested in the model
        real_imgs, _ = batch
        fake_imgs = torch.rand_like(real_imgs) * 2 - 1
        
        inp_imgs = torch.cat([real_imgs, fake_imgs], dim=0)
        real_out, fake_out = self.cnn(inp_imgs).chunk(2, dim=0)
        
        cdiv = fake_out.mean() - real_out.mean()
        self.log('val_contrastive_divergence', cdiv)
        self.log('val_fake_out', fake_out.mean())
        self.log('val_real_out', real_out.mean())

### Callbacks


In [ ]:
class GenerateCallback(pl.Callback):

    def __init__(self, batch_size=8, vis_steps=8, num_steps=256, every_n_epochs=5):
        super().__init__()
        self.batch_size = batch_size         # Number of images to generate
        self.vis_steps = vis_steps           # Number of steps within generation to visualize
        self.num_steps = num_steps           # Number of steps to take during generation
        self.every_n_epochs = every_n_epochs # Only save those images every N epochs (otherwise tensorboard gets quite large)

    def on_epoch_end(self, trainer, pl_module):
        # Skip for all other epochs
        if trainer.current_epoch % self.every_n_epochs == 0:
            # Generate images
            imgs_per_step = self.generate_imgs(pl_module)
            # Plot and add to tensorboard
            for i in range(imgs_per_step.shape[1]):
                step_size = self.num_steps // self.vis_steps
                imgs_to_plot = imgs_per_step[step_size-1::step_size,i]
                grid = torchvision.utils.make_grid(imgs_to_plot, nrow=imgs_to_plot.shape[0], normalize=True, range=(-1,1))
                # grid = torchvision.utils.make_grid(imgs_to_plot, nrow=imgs_to_plot.shape[0], normalize=True)
                trainer.logger.experiment.add_image(f"generation_{i}", grid, global_step=trainer.current_epoch)
                
    def generate_imgs(self, pl_module):
        pl_module.eval()
        start_imgs = torch.rand((self.batch_size,) + pl_module.hparams["img_shape"]).to(pl_module.device)
        start_imgs = start_imgs * 2 - 1
        torch.set_grad_enabled(True)  # Tracking gradients for sampling necessary
        imgs_per_step = Sampler.generate_contour_sgld_samples(pl_module.cnn, start_imgs, steps=self.num_steps, step_size=TRAINING_STEP_SIZE, return_img_per_step=True)
        torch.set_grad_enabled(False)
        pl_module.train()
        return imgs_per_step

In [ ]:
class SamplerCallback(pl.Callback):
    
    def __init__(self, num_imgs=64, every_n_epochs=5):
        super().__init__()
        self.num_imgs = num_imgs             # Number of images to plot
        self.every_n_epochs = every_n_epochs # Only save those images every N epochs (otherwise tensorboard gets quite large)
        
    def on_epoch_end(self, trainer, pl_module):
        if trainer.current_epoch % self.every_n_epochs == 0:
            exmp_imgs = torch.cat(random.choices(pl_module.sampler.examples, k=self.num_imgs), dim=0)
            grid = torchvision.utils.make_grid(exmp_imgs, nrow=8, normalize=True, range=(-1,1))
            # grid = torchvision.utils.make_grid(exmp_imgs, nrow=8, normalize=True)
            trainer.logger.experiment.add_image("sampler", grid, global_step=trainer.current_epoch)

In [ ]:
class OutlierCallback(pl.Callback):
    
    def __init__(self, batch_size=1024):
        super().__init__()
        self.batch_size = batch_size
    
    def on_epoch_end(self, trainer, pl_module):
        with torch.no_grad():
            pl_module.eval()
            rand_imgs = torch.rand((self.batch_size,) + pl_module.hparams["img_shape"]).to(pl_module.device)
            rand_imgs = rand_imgs * 2 - 1.0
            rand_out = pl_module.cnn(rand_imgs).mean()
            pl_module.train()
        
        trainer.logger.experiment.add_scalar("rand_out", rand_out, global_step=trainer.current_epoch)

In [ ]:
class ImageMetricsCallback(pl.Callback):
    
    def __init__(self,  num_imgs=64, every_n_epochs=5):
        super().__init__()
        self.every_n_epochs = every_n_epochs # Only save those images every N epochs (otherwise tensorboard gets quite large)
        self.num_imgs = num_imgs   

    def rescale_img(self, image):
        image = np.clip(image, -1, 1)
        return torch.from_numpy((np.clip(image * 256, 0, 255)).astype(np.uint8))
    
    def on_train_epoch_end(self, trainer, pl_module):
        # images,_ = next(iter(train_loader))
        if trainer.current_epoch % self.every_n_epochs == 0:
            real_imgs = pl_module.last_real_imgs
            fake_imgs = pl_module.last_fake_imgs
            
            #calculate inception score
            if real_imgs.shape[1] == 1:
                real_imgs = torch.cat((real_imgs, real_imgs, real_imgs), dim=1)
                fake_imgs = torch.cat((fake_imgs, fake_imgs, fake_imgs), dim=1)

            real_imgs = self.rescale_img(real_imgs.cpu().numpy())
            fake_imgs = self.rescale_img(fake_imgs.cpu().numpy())

            real_imgs = real_imgs.to(pl_module.device)
            fake_imgs = fake_imgs.to(pl_module.device)

            inception_metrics.update(fake_imgs)
            inception_mean, inception_std = inception_metrics.compute()
            trainer.logger.experiment.add_scalar("inception_mean", inception_mean.item(), global_step=trainer.current_epoch)
            trainer.logger.experiment.add_scalar("inception_std", inception_std.item(), global_step=trainer.current_epoch)

            #calculate FID
            fid_metrics.update(real_imgs, real=True)
            fid_metrics.update(fake_imgs, real=False)
            fid_val = fid_metrics.compute()
            trainer.logger.experiment.add_scalar("fid_value", fid_val.item(), global_step=trainer.current_epoch)
        

### TensorBoard


In [ ]:
# Import tensorboard
%load_ext tensorboard

In [ ]:
# Opens tensorboard in notebook. Adjust the path to your CHECKPOINT_PATH!
%tensorboard --logdir runs

### Running the model


In [ ]:
MAX_NUM_STEPS = 256
if DATASET == 'MNIST':
    data_module = MNISTDataModule()
elif DATASET == 'CIFAR10':
    MAX_NUM_STEPS = 1024
    data_module = CIFAR10DataModule()
else:
    assert False

data_module.prepare_data()
data_module.setup()

In [ ]:
def train_model(**kwargs):
    # Create a PyTorch Lightning trainer with the generation callback
    trainer = pl.Trainer(default_root_dir=os.path.join(CHECKPOINT_PATH, DATASET, DIVERGENCE),
                         gpus=1 if str(device).startswith("cuda") else 0,
                         max_epochs=MAX_EPOCHS,
                         gradient_clip_val=0.1,
                         callbacks=[ModelCheckpoint(save_weights_only=True, mode="min", monitor='val_contrastive_divergence'),
                                    GenerateCallback(every_n_epochs=1),
                                    SamplerCallback(every_n_epochs=1),
                                    OutlierCallback(),
                                    ImageMetricsCallback(every_n_epochs=1),
                                    LearningRateMonitor("epoch")
                                   ],
                        progress_bar_refresh_rate=1)
    # Check whether pretrained model exists. If yes, load it and skip training
    pretrained_filename = os.path.join(CHECKPOINT_PATH, CHECKPOINT_FILE)
    if os.path.isfile(pretrained_filename):
        print("Found pretrained model, loading...")
        model = DeepEnergyModel.load_from_checkpoint(pretrained_filename)
    else:
        pl.seed_everything(42)
        model = DeepEnergyModel(**kwargs)
        trainer.fit(model, data_module)
        model = DeepEnergyModel.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)
    # No testing as we are more interested in other properties
    return model

In [ ]:
model = train_model(divergence=DIVERGENCE,
                    img_shape=data_module.dims, 
                    batch_size=BATCH_SIZE,
                    lr=1e-4,
                    beta1=0.0)

## Analysis

In the last part of the notebook, we will try to take the trained energy-based generative model, and analyse its properties.

### Image Generation


In [ ]:
model.to(device)
pl.seed_everything(43)
callback = GenerateCallback(batch_size=8, vis_steps=8, num_steps=MAX_NUM_STEPS)
imgs_per_step = callback.generate_imgs(model)
imgs_per_step = imgs_per_step.cpu()

The characteristic of sampling with energy-based models is that they require the iterative MCMC algorithm. To gain an insight in how the images change over iterations, we plot a few intermediate samples in the MCMC as well:

In [ ]:
for i in range(imgs_per_step.shape[1]):
    step_size = callback.num_steps // callback.vis_steps
    imgs_to_plot = imgs_per_step[step_size-1::step_size,i]
    imgs_to_plot = torch.cat([imgs_per_step[0:1,i],imgs_to_plot], dim=0)
    grid = torchvision.utils.make_grid(imgs_to_plot, nrow=imgs_to_plot.shape[0], normalize=True, range=(-1,1), pad_value=0.5, padding=2)
    grid = grid.permute(1, 2, 0)
    plt.figure(figsize=(8,8))
    plt.imshow(grid)
    plt.xlabel("Generation iteration")
    plt.xticks([(imgs_per_step.shape[-1]+2)*(0.5+j) for j in range(callback.vis_steps+1)], 
               labels=[1] + list(range(step_size,imgs_per_step.shape[0]+1,step_size)))
    plt.yticks([])
    plt.show()

### Out-of-distribution detection

A very common and strong application of energy-based models is out-of-distribution detection (sometimes referred to as "anomaly" detection). As more and more deep learning models are applied in production and applications, a crucial aspect of these models is to know what the models don't know. Deep learning models are usually overconfident, meaning that they classify even random images sometimes with 100% probability. Clearly, this is not something that we want to see in applications. Energy-based models can help with this problem because they are trained to detect images that do not fit the training dataset distribution. Thus, in those applications, you could train an energy-based model along with the classifier, and only output predictions if the energy-based models assign a (unnormalized) probability higher than $\delta$ to the image. You can actually combine classifiers and energy-based objectives in a single model, as proposed in this [paper](https://arxiv.org/abs/1912.03263).

In this part of the analysis, we want to test the out-of-distribution capability of our energy-based model. Remember that a lower output of the model denotes a low probability. Thus, we hope to see low scores if we enter random noise to the model:

In [ ]:
with torch.no_grad():
    rand_imgs = torch.rand((128,) + model.hparams.img_shape).to(model.device)
    rand_imgs = rand_imgs * 2 - 1.0
    rand_out = model.cnn(rand_imgs).mean()
    print(f"Average score for random images: {rand_out.item():4.2f}")

As we hoped, the model assigns very low probability to those noisy images. As another reference, let's look at predictions for a batch of images from the training set:

In [ ]:
with torch.no_grad():
    train_imgs,_ = next(iter(data_module.val_dataloader()))
    train_imgs = train_imgs.to(model.device)
    train_out = model.cnn(train_imgs).mean()
    print(f"Average score for training images: {train_out.item():4.2f}")

The scores are close to 0 because of the regularization objective that was added to the training. So clearly, the model can distinguish between noise and real digits. However, what happens if we change the training images a little, and see which ones gets a very low score?

In [ ]:
@torch.no_grad()
def compare_images(img1, img2):
    imgs = torch.stack([img1, img2], dim=0).to(model.device)
    score1, score2 = model.cnn(imgs).cpu().chunk(2, dim=0)
    # grid = torchvision.utils.make_grid([img1.cpu(), img2.cpu()], nrow=2, normalize=True, range=(-1,1), pad_value=0.5, padding=2)
    grid = torchvision.utils.make_grid([img1.cpu(), img2.cpu()], nrow=2, normalize=True, pad_value=0.5, padding=2)
    grid = grid.permute(1, 2, 0)
    plt.figure(figsize=(4,4))
    plt.imshow(grid)
    plt.xticks([(img1.shape[2]+2)*(0.5+j) for j in range(2)],
               labels=["Original image", "Transformed image"])
    plt.yticks([])
    plt.show()
    print("Score original image: %4.2f" % score1)
    print("Score transformed image: %4.2f" % score2)

We use a random test image for this. Feel free to change it to experiment with the model yourself.

In [ ]:
test_imgs, _ = next(iter(dm.val_dataloader()))
exmp_img = test_imgs[0].to(model.device)

The first transformation is to add some random noise to the image:

In [ ]:
img_noisy = exmp_img + torch.randn_like(exmp_img) * 0.3
img_noisy.clamp_(min=-1.0, max=1.0)
compare_images(exmp_img, img_noisy)

We can see that the score considerably drops. Hence, the model can detect random Gaussian noise on the image. This is also to expect as initially, the "fake" samples are pure noise images.

Next, we flip an image and check how this influences the score:

In [ ]:
img_flipped = exmp_img.flip(dims=(1,2))
compare_images(exmp_img, img_flipped)

If the digit can only be read in this way, for example, the 7, then we can see that the score drops. However, the score only drops slightly. This is likely because of the small size of our model. Keep in mind that generative modeling is a much harder task than classification, as we do not only need to distinguish between classes but learn **all** details/characteristics of the digits. With a deeper model, this could eventually be captured better (but at the cost of greater training instability).

Finally, we check what happens if we reduce the digit significantly in size:

In [ ]:
img_tiny = torch.zeros_like(exmp_img)-1
img_tiny[:,exmp_img.shape[1]//2:,exmp_img.shape[2]//2:] = exmp_img[:,::2,::2]
compare_images(exmp_img, img_tiny)

The score again drops but not by a large margin, although digits in the MNIST dataset usually are much larger. 

Overall, we can conclude that our model is good for detecting Gaussian noise and smaller transformations to existing digits. Nonetheless, to obtain a very good out-of-distribution model, we would need to train deeper models and for more iterations.

### Instability

Finally, we should discuss the possible instabilities of energy-based models, in particular for the example of image generation that we have implemented in this notebook. In the process of hyperparameter search for this notebook, there have been several models that diverged. Divergence in energy-based models means that the models assign a high probability to examples of the training set which is a good thing. However, at the same time, the sampling algorithm fails and only generates noise images that obtain minimal probability scores. This happens because the model has created many local maxima in which the generated noise images fall. The energy surface over which we calculate the gradients to reach data points with high probability has "diverged" and is not useful for our MCMC sampling.

Besides finding the optimal hyperparameters, a common trick in energy-based models is to reload stable checkpoints. If we detect that the model is diverging, we stop the training, load the model from one epoch ago where it did not diverge yet. Afterward, we continue training and hope that with a different seed the model is not diverging again. Nevertheless, this should be considered as the "last hope" for stabilizing the models, and careful hyperparameter tuning is the better way to do so. Sensitive hyperparameters include `step_size`, `steps` and the noise standard deviation in the sampler, and the learning rate and feature dimensionality in the CNN model. 

## Conclusion

In this tutorial, we have discussed energy-based models for generative modeling. The concept relies on the idea that any strictly positive function can be turned into a probability distribution by normalizing over the whole dataset. As this is not reasonable to calculate for high dimensional data like images, we train the model using contrastive divergence and sampling via MCMC. While the idea allows us to turn any neural network into an energy-based model, we have seen that there are multiple training tricks needed to stabilize the training. Furthermore, the training time of these models is relatively long as, during every training iteration, we need to sample new "fake" images, even with a sampling buffer. In the next lectures and assignment, we will see different generative models (e.g. VAE, GAN, NF) that allow us to do generative modeling more stably, but with the cost of more parameters.